In [1]:
import numpy as np
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import seaborn as sns
import pycountry_convert as pc

from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim

In [2]:
# Исходные данные на странице
#
# https://openflights.org/data.html
# 
url_airport = 'https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat'
url_routers = 'https://raw.githubusercontent.com/jpatokal/openflights/master/data/routes.dat'

df_raw_airport = pd.read_csv(url_airport)
display(df_raw_airport.head(3))
df_raw_routers = pd.read_csv(url_routers)
display(df_raw_routers.head(3))

,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081689834590001,145.391998291,5282,10,U,Pacific/Port_Moresby,airport,OurAirports
0,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports
1,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports
2,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports


,2B,410,AER,2965,KZN,2990,Unnamed: 6,0,CR2
0,2B,410,ASF,2966,KZN,2990,NaN,0,CR2
1,2B,410,ASF,2966,MRV,2962,NaN,0,CR2
2,2B,410,CEK,2968,KZN,2990,NaN,0,CR2


In [3]:
columns_airport = ['Airport ID', 'Name', 'City',
                   'Country', 'IATA', 'ICAO', 'Latitude', 'Longitude', 'Altitude', 'Timezone',
                   'DST', 'Tz database time zone', 'Type', 'Source']

columns_routers = ['Airline', 'Airline ID', 'Source airport', 'Source airport ID', 
                   'Destination airport', 'Destination airport ID', 'Codeshare', 'Stops', 'Equipment']

df_raw_airport = pd.read_csv(url_airport, names = columns_airport)
display(df_raw_airport.head(3))
df_raw_routers = pd.read_csv(url_routers, names = columns_routers)
display(df_raw_routers.head(3))

,Airport ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,Tz database time zone,Type,Source
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.08169,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.20708,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.82679,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports


,Airline,Airline ID,Source airport,Source airport ID,Destination airport,Destination airport ID,Codeshare,Stops,Equipment
0,2B,410,AER,2965,KZN,2990,NaN,0,CR2
1,2B,410,ASF,2966,KZN,2990,NaN,0,CR2
2,2B,410,ASF,2966,MRV,2962,NaN,0,CR2


# Задание 1

Добавить столбец <code>Continent</code> с данными, которые будут содержать название континента.

In [4]:
#Создаем функцию, которая будет конвертировать название страны в континент, к которому она принадлежит
def tryconvert(x):
    continents = {
    'NA': 'North America',
    'SA': 'South America', 
    'AS': 'Asia',
    'OC': 'Australia',
    'AF': 'Africa',
    'EU': 'Europe'
    }
    try:
        return continents[pc.country_alpha2_to_continent_code(pc.country_name_to_country_alpha2(x, cn_name_format="default"))]
    except (KeyError, TypeError):
        return np.nan
df_raw_airport['Continent'] = df_raw_airport['Country'].apply(tryconvert)

In [5]:
df_raw_airport['Continent'].unique()

array(['Australia', 'North America', 'Europe', 'Africa', nan,
       'South America', 'Asia'], dtype=object)

In [6]:
list(df_raw_airport[df_raw_airport['Continent'].isnull()]['Country'].unique())

["Cote d'Ivoire",
 'Congo (Brazzaville)',
 'Congo (Kinshasa)',
 'Saint Helena',
 'Reunion',
 'Antarctica',
 'West Bank',
 'Midway Islands',
 'Virgin Islands',
 'Netherlands Antilles',
 'Burma',
 'East Timor',
 'Johnston Atoll',
 'Western Sahara',
 'Wake Island']

In [7]:
#Создаем словарь из стран, названия которых устарели и соотносим их с континентом
country_dict ={"Cote d'Ivoire": 'Africa',
               'Congo (Brazzaville)': 'Africa',
               'Congo (Kinshasa)': 'Africa',
               'Saint Helena': 'Africa',
               'Reunion': 'Africa',
               'Antarctica': 'Antarctica',
               'West Bank': 'Asia',
               'Midway Islands': 'North America',
               'Virgin Islands': 'North America',
               'Netherlands Antilles':'Europe',
               'Burma': 'Asia',
               'East Timor': 'North America',
               'Johnston Atoll':'North America',
               'Western Sahara': 'Africa',
               'Wake Island': 'North America'}

In [8]:
df_raw_airport['Continent'] = df_raw_airport.apply(lambda x: country_dict[x['Country']] if x['Country'] in country_dict.keys() 
                                                   else x['Continent'], axis=1)

In [9]:
print(df_raw_airport['Continent'].unique())
df_raw_airport

['Australia' 'North America' 'Europe' 'Africa' 'South America' 'Asia'
 'Antarctica']


,Airport ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,Tz database time zone,Type,Source,Continent
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports,Australia
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports,Australia
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports,Australia
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports,Australia
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10,U,Pacific/Port_Moresby,airport,OurAirports,Australia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7693,14106,Rogachyovo Air Base,Belaya,Russia,\N,ULDA,71.616699,52.478298,272,\N,\N,\N,airport,OurAirports,Europe
7694,14107,Ulan-Ude East Airport,Ulan Ude,Russia,\N,XIUW,51.849998,107.737999,1670,\N,\N,\N,airport,OurAirports,Europe
7695,14108,Krechevitsy Air Base,Novgorod,Russia,\N,ULLK,58.625000,31.385000,85,\N,\N,\N,airport,OurAirports,Europe
7696,14109,Desierto de Atacama Airport,Copiapo,Chile,CPO,SCAT,-27.261200,-70.779198,670,\N,\N,\N,airport,OurAirports,South America


# Задание 2

Из списка континентов случайно выбрать континент и срезать таблицу по названию континента и аэропорту имеющему код IATA (имеется в виду - для выбранного континента только аэропорты с кодом IATA). Каждый раз при выполнении кода континент случайно выбирается.

In [10]:
#Здесь нам поможет функция .sample(), которая возвращает случайную выборку элементов из оси объекта
cont_random = list(df_raw_airport['Continent'].sample())[0]
print(cont_random)
df_raw_airport[(df_raw_airport['IATA'] != r"\N") & (df_raw_airport['Continent'] == cont_random)]

Australia


,Airport ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,Tz database time zone,Type,Source,Continent
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports,Australia
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports,Australia
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports,Australia
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports,Australia
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10,U,Pacific/Port_Moresby,airport,OurAirports,Australia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7607,13628,Raroia Airport,Raroia,French Polynesia,RRR,NTKO,-16.045000,-142.476944,18,-10,N,\N,airport,OurAirports,Australia
7608,13629,Kauehi Airport,Kauehi,French Polynesia,KHZ,NTKA,-15.780800,-145.123993,13,-10,N,\N,airport,OurAirports,Australia
7609,13630,Tatakoto Airport,Tatakoto,French Polynesia,TKV,NTGO,-17.355301,-138.445007,12,-10,N,\N,airport,OurAirports,Australia
7610,13631,Sasereme Airport,Sasereme,Papua New Guinea,TDS,AYSS,-7.621700,142.868000,121,10,N,\N,airport,OurAirports,Australia


# Задание 3

На основе таблицы связи аэропортов создать таблицу с кодом аэропорта, количеством взлетов и количеством посадок.

In [11]:

df_raw_routers

,Airline,Airline ID,Source airport,Source airport ID,Destination airport,Destination airport ID,Codeshare,Stops,Equipment
0,2B,410,AER,2965,KZN,2990,NaN,0,CR2
1,2B,410,ASF,2966,KZN,2990,NaN,0,CR2
2,2B,410,ASF,2966,MRV,2962,NaN,0,CR2
3,2B,410,CEK,2968,KZN,2990,NaN,0,CR2
4,2B,410,CEK,2968,OVB,4078,NaN,0,CR2
...,...,...,...,...,...,...,...,...,...
67658,ZL,4178,WYA,6334,ADL,3341,NaN,0,SF3
67659,ZM,19016,DME,4029,FRU,2912,NaN,0,734
67660,ZM,19016,FRU,2912,DME,4029,NaN,0,734
67661,ZM,19016,FRU,2912,OSS,2913,NaN,0,734


In [12]:
#Проверка формата значений некоторых колонок
print(type(df_raw_routers['Source airport ID'][5]))
print(type(df_raw_routers['Destination airport ID'][5]))
print(type(df_raw_routers['Codeshare'][5]))
print(df_raw_routers['Codeshare'].unique())
print(df_raw_routers['Codeshare'].value_counts())

#Изменение формата значений, чтобы можно было мёрджить таблицы
df_raw_routers.replace({r'\N': np.nan}, inplace = True)
df_raw_routers['Source airport ID'] = pd.to_numeric(df_raw_routers['Source airport ID'])
df_raw_routers['Destination airport ID'] = pd.to_numeric(df_raw_routers['Destination airport ID'])

<class 'str'>
<class 'str'>
<class 'float'>
[nan 'Y']
Y    14597
Name: Codeshare, dtype: int64


In [15]:
#добавляем к таблице с аэропорта колонку с ID аэропорта вылета/прилёта, чтобы можно было считать вылеты/прилёты 
# по коду аэропорта IATA

# вылеты (взлетели)
took_off = df_raw_airport.merge(df_raw_routers['Source airport ID'], left_on='Airport ID', right_on='Source airport ID', how='left')

# прилёты (приземлились)
landed = df_raw_airport.merge(df_raw_routers['Destination airport ID'], left_on='Airport ID', right_on='Destination airport ID', how='left')

In [16]:
#У меня до этого неправильно считались вылеты и прилёты, поэтому я здесь проверял, сюда можно не обращать внимания
#display(took_off.head(5))

zzv = landed[landed["IATA"] == 'ZZV']
zzv

,Airport ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,Tz database time zone,Type,Source,Continent,Destination airport ID
71066,11868,Zanesville Municipal Airport,Zanesville,United States,ZZV,KZZV,39.944401,-81.892097,900,\N,\N,\N,airport,OurAirports,North America,NaN


In [17]:
#Создание искомой таблицы, где took_off - количество вылетов, landed - количество прилётов. Мёрджим по коду 
# аэропортов IATA. Сначала пробовал считать по любому столбцу, но тогда считало все записи, даже если 
# прилёта/вылета не было, поэтому функцию count запускаем только для ID аэропорта источника или назначения, тк там
# есть значения NaN, что значит что вылета/прилёта не было

table3 = took_off.groupby('IATA', as_index=False).agg({'Source airport ID':'count'})\
                    .merge(landed.groupby('IATA', as_index=False).agg({'Destination airport ID':'count'}), on='IATA')\
                    .rename(columns = {'Source airport ID':'took-off', 'Destination airport ID':'landed'})

table3

,IATA,took-off,landed
0,AAA,0,0
1,AAC,0,0
2,AAE,9,9
3,AAF,0,0
4,AAH,0,0
...,...,...,...
6068,ZYI,15,15
6069,ZYL,4,6
6070,ZZU,0,0
6071,ZZV,0,0


# Задание 4

Используя в качестве ключа код аэропорта слить данные для отобранного континента и собрать таблицу - страна, название аэропорта, количество взлет/посадок (сумма взлетов и посадок)

In [18]:
table4 = df_raw_airport[df_raw_airport['Continent'] == cont_random].merge(table3, how='left', on='IATA')\
                        .groupby(['Country', 'Name'], as_index=False).agg({'took-off': 'sum', 'landed': 'sum'})

table4['took-off_landed'] = table4['took-off'] + table4['landed']
table4.drop(['took-off', 'landed'], axis=1, inplace=True)

table4

,Country,Name,took-off_landed
0,American Samoa,Fitiuta Airport,0
1,American Samoa,Pago Pago International Airport,2
2,Australia,Adelaide International Airport,102
3,Australia,Adelaide Parafield Airport,354
4,Australia,Albany Airport,6
...,...,...,...
611,Vanuatu,Uléi Airport,5
612,Vanuatu,Valesdir Airport,2
613,Vanuatu,Walaha Airport,2
614,Wallis and Futuna,Hihifo Airport,6


# Задание 5

1. Найти коды аэропортов для которых нет взлет/посадок в таблице аэропортов.
2. Найти коды аэропортов для которых нет названий в таблице связей аэропортов.

In [19]:
# Задание 5.1
# Смёрджим к табличке аэропортов колонку ID аэропорта вылета/прилёта и авиакомпании (где нет вылета, будет пусто)

took_off_empty = df_raw_airport.merge(df_raw_routers[['Source airport ID', 'Airline']], left_on='Airport ID', 
                                    right_on='Source airport ID', how='left')

landed_empty = df_raw_airport.merge(df_raw_routers[['Destination airport ID', 'Airline']], left_on='Airport ID', 
                                    right_on='Destination airport ID', how='left')



In [20]:
took_off_empty.head(2)

,Airport ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,Tz database time zone,Type,Source,Continent,Source airport ID,Airline
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.08169,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports,Australia,1.0,CG
1,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.08169,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports,Australia,1.0,CG


In [21]:
landed_empty.head(2)

,Airport ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,Tz database time zone,Type,Source,Continent,Destination airport ID,Airline
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.08169,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports,Australia,1.0,CG
1,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.08169,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports,Australia,1.0,CG


In [22]:
took_off_landed_empty = set(took_off_empty[pd.isnull(took_off_empty['Airline'])]['IATA'].unique())\
    & set(landed_empty[pd.isnull(landed_empty['Airline'])]['IATA'].unique()) 

In [23]:
# Коды аэропортов, для которых нет взлётов/посадок в таблице

took_off_landed_empty

{'BXG',
 'DRA',
 'FRG',
 'IDP',
 'HSZ',
 'MQU',
 'CLP',
 'SGQ',
 'NIS',
 'MVQ',
 'MCJ',
 'HHZ',
 'POE',
 'LGH',
 'PCF',
 'IEJ',
 'HLE',
 'WUN',
 'MGC',
 'GOB',
 'STP',
 'CYI',
 'DBB',
 'WIO',
 'BSU',
 'BNU',
 'JSK',
 'LVK',
 'PCD',
 'HUQ',
 'POX',
 'YRO',
 'CXP',
 'YVG',
 'AVB',
 'ADM',
 'HSM',
 'PHN',
 'CCA',
 'ELY',
 'VQQ',
 'LNL',
 'DAA',
 'TCH',
 'YMH',
 'NNA',
 'HAR',
 'STB',
 'NSE',
 'FAC',
 'UAR',
 'WTS',
 'SHY',
 'AEH',
 'KDL',
 'IYK',
 'PTZ',
 'TGN',
 'BCT',
 'KAU',
 'MFQ',
 'BPE',
 'AEU',
 'CCL',
 'ANP',
 'ERG',
 'YQI',
 'MNZ',
 'AMN',
 'UGU',
 'LMR',
 'NGF',
 'WOS',
 'KLZ',
 'SFK',
 'CNR',
 'GLZ',
 'MXN',
 'AKR',
 'ZMM',
 'POT',
 'KDT',
 'SEY',
 'KFE',
 'VCD',
 'CKT',
 'YAI',
 'IRD',
 'VOK',
 'SXN',
 'VLR',
 'LFM',
 'NZJ',
 'MGY',
 'SNS',
 'YKH',
 'HYV',
 'KGG',
 'ALX',
 'GQQ',
 'MVV',
 'CFG',
 'DTI',
 'YSH',
 'RHP',
 'LIQ',
 'ORP',
 'YVA',
 'BTL',
 'NVS',
 'TXG',
 'YQV',
 'WAQ',
 'RMY',
 'MIU',
 'AYO',
 'ATO',
 'AYK',
 'BJW',
 'XSI',
 'LID',
 'RTC',
 'ZZV',
 'SPF',
 'SSC',


In [24]:
# Место для отдыха глаз
#
#
#
#
#
#
# Конец места для отдыха глаз

In [25]:
# Задание 5.2
# Смёрджим к табличке маршрутов колонку ID аэропорта вылета/прилёта и названия аэропорта

took_off_names_empty = df_raw_routers.merge(df_raw_airport[['Airport ID', 'Name']], left_on='Source airport ID', 
                                    right_on='Airport ID', how='left')

landed_names_empty = df_raw_routers.merge(df_raw_airport[['Airport ID', 'Name']], left_on='Destination airport ID', 
                                    right_on='Airport ID', how='left')

In [26]:
took_off_names_empty.head(2)

,Airline,Airline ID,Source airport,Source airport ID,Destination airport,Destination airport ID,Codeshare,Stops,Equipment,Airport ID,Name
0,2B,410,AER,2965.0,KZN,2990.0,NaN,0,CR2,2965.0,Sochi International Airport
1,2B,410,ASF,2966.0,KZN,2990.0,NaN,0,CR2,2966.0,Astrakhan Airport


In [27]:
landed_names_empty.head(2)

,Airline,Airline ID,Source airport,Source airport ID,Destination airport,Destination airport ID,Codeshare,Stops,Equipment,Airport ID,Name
0,2B,410,AER,2965.0,KZN,2990.0,NaN,0,CR2,2990.0,Kazan International Airport
1,2B,410,ASF,2966.0,KZN,2990.0,NaN,0,CR2,2990.0,Kazan International Airport


In [28]:
names_empty = set(took_off_names_empty[pd.isnull(took_off_names_empty['Name'])]['Source airport'].unique()) |\
            set(landed_names_empty[pd.isnull(landed_names_empty['Name'])]['Destination airport'].unique())

In [29]:
# Коды аэропортов для которых нет названий в таблице

names_empty

{'ABB',
 'ACU',
 'AOQ',
 'AOS',
 'ATT',
 'BFQ',
 'BLD',
 'BNY',
 'BSS',
 'BTC',
 'BUL',
 'BVV',
 'CBS',
 'CFK',
 'CGA',
 'CKD',
 'CKX',
 'CUK',
 'CXF',
 'CZH',
 'CZJ',
 'DEF',
 'DGA',
 'DHB',
 'DIW',
 'DOB',
 'EAR',
 'EBH',
 'EDA',
 'EKS',
 'ELL',
 'EXI',
 'FBS',
 'FDE',
 'FSZ',
 'FVM',
 'GCH',
 'GCW',
 'GHE',
 'GLN',
 'GMO',
 'GNU',
 'GSM',
 'HOE',
 'HYD',
 'HYL',
 'IGM',
 'IKE',
 'INB',
 'IOQ',
 'IOT',
 'IUI',
 'JAU',
 'JGS',
 'JJA',
 'JKR',
 'JUK',
 'JUU',
 'KAE',
 'KBC',
 'KBU',
 'KCC',
 'KCG',
 'KCK',
 'KCL',
 'KCO',
 'KCQ',
 'KGQ',
 'KGX',
 'KHQ',
 'KHY',
 'KHZ',
 'KKB',
 'KKI',
 'KLL',
 'KOZ',
 'KPB',
 'KPR',
 'KQT',
 'KSE',
 'KTB',
 'KUS',
 'KUZ',
 'KWF',
 'KXU',
 'KZB',
 'LBP',
 'LCR',
 'LFM',
 'LGQ',
 'LKE',
 'LLB',
 'LMA',
 'LMC',
 'LNV',
 'LSA',
 'MCM',
 'MEU',
 'MJU',
 'MLH',
 'MNS',
 'MNT',
 'MPP',
 'MQJ',
 'MRA',
 'MTE',
 'MZW',
 'NER',
 'NGK',
 'NIQ',
 'NKI',
 'NOP',
 'NSQ',
 'NTG',
 'NUP',
 'OGM',
 'OHH',
 'OLH',
 'OLL',
 'ORI',
 'ORU',
 'ORZ',
 'OSM',
 'OTD',
 'PBZ',


# Задание 6

Для таблицы в задании 4, для трех крупнейших аэропортов для каждой страны, собрать ряд - первый индекс: название страны, второй индекс: название аэропорта, значение: количество взлет посадок. Первый индекс упорядочен по названию страны (по алфавиту), а назание аэропорта по количеству взлет посадок от большего к меньшему.

In [30]:
table4.head()

,Country,Name,took-off_landed
0,American Samoa,Fitiuta Airport,0
1,American Samoa,Pago Pago International Airport,2
2,Australia,Adelaide International Airport,102
3,Australia,Adelaide Parafield Airport,354
4,Australia,Albany Airport,6


In [31]:
display(table4.groupby(['Country'])['took-off_landed'].nlargest(3).reset_index())

,Country,level_1,took-off_landed
0,American Samoa,1,2
1,American Samoa,0,0
2,Australia,283,410
3,Australia,106,357
4,Australia,3,354
5,Cook Islands,341,27
6,Cook Islands,335,5
7,Cook Islands,336,3
8,Fiji,353,83
9,Fiji,354,23


In [32]:
# Функция nlargest() возвращает первые n строк, упорядоченных по столбцам в порядке убывания.

table4_sorted = (table4.groupby(['Country'])['took-off_landed'].nlargest(3).reset_index().\
                merge(table4['Name'], left_on='level_1', right_on = table4.index).drop('level_1', axis=1)).\
                sort_values(['Country', 'took-off_landed'], ascending=[True, False]).\
                set_index(['Country', 'Name']) 

# По строкам:
# Группируем по странам и делаем выборку 3 строк для каждой страны с наибольш. кол-вом действий
# Добавляем по первоначальному индексу название аэропорта
## Упорядочиваем страны по алфавиту, количество действий в аэропорту от большего к меньшему
# первый индекс: название страны, второй индекс: название аэропорта

In [33]:
# Итоговая таблица
table4_sorted

took-off_landed
Country                  Name                                                         
American Samoa           Pago Pago International Airport                             2
                         Fitiuta Airport                                             0
Australia                Sydney Kingsford Smith International Airport              410
                         Flinders Island Airport                                   357
                         Adelaide Parafield Airport                                354
Cook Islands             Rarotonga International Airport                            27
                         Aitutaki Airport                                            5
                         Enua Airport                                                3
Fiji                     Nadi International Airport                                 83
                         Nausori International Airport                              23
                         Matei Airport                                               6
French Polynesia         Faa'a International Airport                                77
                         Bora Bora Airport                                          12
                         Rangiroa Airport                                           11
Guam                     Antonio B. Won Pat International Airport                   70
                         Andersen Air Force Base                                     0
Kiribati                 Cassidy International Airport                               8
                         Bonriki International Airport                               6
                         Abaiang Airport                                             0
Marshall Islands         Dyess Army Air Field                                      354
                         Marshall Islands International Airport                      8
                         Bucholz Army Air Field                                      6
Micronesia               Pohnpei International Airport                               6
                         Chuuk International Airport                                 4
                         Kosrae International Airport                                4
Nauru                    Nauru International Airport                                 6
New Caledonia            La Tontouta International Airport                          26
                         Nouméa Magenta Airport                                     17
                         Lifou Airport                                               6
New Zealand              Glacier Country Heliport                                  354
                         Haast Aerodrome                                           354
                         Hastings Aerodrome                                        354
Niue                     Niue International Airport                                  1
Norfolk Island           Norfolk Island International Airport                        5
Northern Mariana Islands Saipan International Airport                               20
                         Rota International Airport                                  4
                         Tinian International Airport                                0
Palau                    Babelthuap Airport                                         19
Papua New Guinea         Port Moresby Jacksons International Airport               102
                         Tokua Airport                                              25
                         Mount Hagen Kagamuga Airport                               22
Samoa                    Faleolo International Airport                              11
                         Fagali'i Airport                                            0
                         Maota Airport                                               0
Solomon Islands          Honiara International Airport               

# Задание 7

Для каждого континента отобрать не более 3 стран с наибольшим количеством взлет посадок и нанести их на географическую карту. 

In [34]:
table7 = df_raw_airport.merge(table3, how='left', on='IATA')\
                        .groupby(['Country'], as_index=False)\
                        .agg({'took-off': 'sum', 'landed': 'sum'})
table7['took_off_landed'] = table7['took-off'] + table7['landed']
table7.drop(['took-off', 'landed'], axis=1, inplace=True)

table7_cont = df_raw_airport[['Continent', 'Country']].drop_duplicates().merge(table7, how='left', on='Country')


top3_for_cont = table7_cont.groupby(['Continent'])['took_off_landed'].nlargest(3).reset_index()\
                                    .merge(table7_cont['Country'], left_on='level_1', 
                                        right_on = table7_cont.index)\
                                    .drop('level_1', axis=1)     


top3_for_cont.head()

,Continent,took_off_landed,Country
0,Africa,14695,South Africa
1,Africa,4295,Kenya
2,Africa,2547,Uganda
3,Antarctica,2124,Antarctica
4,Asia,18417,China


In [37]:
# Находим координаты стран с помощью geopy
longitude = []
latitude = []

def findGeocode(country):
    try:
        geolocator = Nominatim(user_agent="Aero_Shamov")
        return geolocator.geocode(country)
    except GeocoderTimedOut:
        return findGeocode(country)    
  
for i in (top3_for_cont["Country"]):
    if findGeocode(i) != None:
        loc = findGeocode(i)
        latitude.append(loc.latitude)
        longitude.append(loc.longitude)
    else:
        latitude.append(np.nan)
        longitude.append(np.nan)
        
top3_for_cont["Latitude"] = latitude
top3_for_cont["Longitude"] = longitude

top3_for_cont.head()

,Continent,took_off_landed,Country,Latitude,Longitude
0,Africa,14695,South Africa,-28.816624,24.991639
1,Africa,4295,Kenya,1.441968,38.431398
2,Africa,2547,Uganda,1.533355,32.216658
3,Antarctica,2124,Antarctica,-79.406307,0.314931
4,Asia,18417,China,35.000074,104.999927


In [38]:
# Нанесение стран на карту
fig = go.Figure(go.Scattermapbox(lat=top3_for_cont['Latitude'], 
                                 lon=top3_for_cont['Longitude'], 
                                 text=top3_for_cont['Country'],
                                 marker=dict(colorbar=dict(title="Кол-во взлёт/посадок"),
                                             color=top3_for_cont['took_off_landed']
                                             #size=top5_for_cont_coord['took_off_landed'] /50)
                                )))

title7 = 'Страны с наибольшим количеством взлётов/посадок для каждого континента'

fig.update_layout(title = title7, mapbox_style="open-street-map")

fig.show()

# Задание 8

Для каждого континента отобрать 5 наиболее крупных аэропортов с наибольшим количеством взлет посадок и нанести их на географическую карту. 

In [39]:
table8 = df_raw_airport.merge(table3, how='left', on='IATA')\
                   .groupby(['Continent', 'Name'], as_index=False)\
                   .agg({'took-off': 'sum', 'landed': 'sum'})

table8['took_off_landed'] = table8['took-off'] + table8['landed']
table8.drop(['took-off', 'landed'], axis=1, inplace=True)

top5_for_cont = table8.groupby(['Continent'])['took_off_landed'].nlargest(5).reset_index()\
                                    .merge(table8['Name'], left_on='level_1', 
                                        right_on = table8.index)\
                                    .drop('level_1', axis=1)    

top5_for_cont_coord = top5_for_cont.merge(df_raw_airport[['Name', 'Latitude','Longitude']], 
                                                left_on='Name', right_on = 'Name')


top5_for_cont_coord['Latitude'] = pd.to_numeric(top5_for_cont_coord['Latitude'])
top5_for_cont_coord['Longitude'] = pd.to_numeric(top5_for_cont_coord['Longitude'])

top5_for_cont_coord.head()

,Continent,took_off_landed,Name,Latitude,Longitude
0,Africa,354,Ain Oussera Airport,35.525398,2.878710
1,Africa,354,Aliwal North Airport,-30.680000,26.730000
2,Africa,354,Alkantpan Copper Airport,-29.906401,22.316700
3,Africa,354,Almaza Air Force Base,30.091801,31.360001
4,Africa,354,Alphonse Airport,-7.004780,52.726200


In [40]:
# Нанесение аэропортов на карту
fig = go.Figure(go.Scattermapbox(lat=top5_for_cont_coord['Latitude'], 
                                 lon=top5_for_cont_coord['Longitude'], 
                                 text=top5_for_cont_coord['Name'],
                                 marker=dict(colorbar=dict(title="Кол-во взлёт/посадок"),
                                             color=top5_for_cont_coord['took_off_landed']
                                             #size=top5_for_cont_coord['took_off_landed'] /50)
                                )))

title8 = '5 наиболее крупных аэропортов с наибольшим количеством взлётов/посадок для каждого континента'

fig.update_layout(title = title8,mapbox_style="open-street-map")

fig.show()

# Задание 9

Для наиболее крупных аэропортов (задание 8) построить маршрутную транспортную сеть и нанести их на географическую карту, линия должна отображать интенсивность взаимосвязи (прозрачность, толщина). 

In [41]:
#top5_for_cont.head(2)

#Нам вновь нужны все данные из таблицы аэропортов, но только для самых крупных из таблицы в 8 задании

airport_top = df_raw_airport[df_raw_airport['Name'].isin(top5_for_cont['Name'])]


#Все прилёты во все топовые аэропорты и направления
destination_top = df_raw_routers[df_raw_routers['Destination airport'].isin(airport_top['IATA'])]

top_landed = destination_top[destination_top['Source airport'].isin(airport_top['IATA'])]

top_landed

,Airline,Airline ID,Source airport,Source airport ID,Destination airport,Destination airport ID,Codeshare,Stops,Equipment
3482,9C,15999,PVG,3406.0,SIN,3316.0,NaN,0,320
3518,9C,15999,SIN,3316.0,PVG,3406.0,NaN,0,320
4683,AA,24,AMS,580.0,LHR,507.0,Y,0,320 767 321 319
4699,AA,24,ATL,3682.0,DFW,3670.0,NaN,0,M80 319 M83
4700,AA,24,ATL,3682.0,LHR,507.0,Y,0,777
...,...,...,...,...,...,...,...,...,...
63456,WN,4547,ATL,3682.0,LAX,3484.0,NaN,0,73W
63969,WN,4547,LAX,3484.0,ATL,3682.0,NaN,0,73W
64569,WS,5416,ATL,3682.0,JFK,3797.0,Y,0,738
67114,ZH,4611,PEK,3364.0,PVG,3406.0,NaN,0,330 737


In [42]:
top_routes = top_landed.groupby(['Source airport', 'Destination airport'], as_index=False)['Airline'].count()

In [43]:
top_routes.head()

,Source airport,Destination airport,Airline
0,AMS,ATL,2
1,AMS,BCN,5
2,AMS,CDG,3
3,AMS,DFW,2
4,AMS,FRA,4


In [44]:
top_routes1 = top_routes.fillna(0).rename(columns={'Airline': 'Intensity'})

top_routes1['Intensity'] = pd.to_numeric(top_routes1['Intensity'])

top_routes1 = top_routes1.merge(airport_top[['IATA', 'Latitude','Longitude']], 
                                left_on='Source airport', right_on='IATA', how='left').\
                            rename(columns={'Latitude':'Latitude_S', 'Longitude':'Longitude_S'})

top_routes1 = top_routes1.merge(airport_top[['IATA', 'Latitude','Longitude']], 
                                left_on='Destination airport', right_on='IATA', how='left').\
                            rename(columns={'Latitude':'Latitude_D', 'Longitude':'Longitude_D'}).\
                            drop(['IATA_x', 'IATA_y'], axis=1)

top_routes1

,Source airport,Destination airport,Intensity,Latitude_S,Longitude_S,Latitude_D,Longitude_D
0,AMS,ATL,2,52.308601,4.763890,33.636700,-84.428101
1,AMS,BCN,5,52.308601,4.763890,41.297100,2.078460
2,AMS,CDG,3,52.308601,4.763890,49.012798,2.550000
3,AMS,DFW,2,52.308601,4.763890,32.896801,-97.038002
4,AMS,FRA,4,52.308601,4.763890,50.033333,8.570556
...,...,...,...,...,...,...,...
176,SYD,ICN,4,-33.946098,151.177002,37.469101,126.450996
177,SYD,LAX,6,-33.946098,151.177002,33.942501,-118.407997
178,SYD,PEK,1,-33.946098,151.177002,40.080101,116.584999
179,SYD,PVG,4,-33.946098,151.177002,31.143400,121.805000


In [45]:
type(top_routes1['Intensity'][3])

numpy.int64

In [46]:
# Нанесение крупнейших аэропортов и интенсивности связи между ними на карту
fig = go.Figure(go.Scattermapbox(lat=top_routes1['Latitude_S'], 
                                 lon=top_routes1['Longitude_S'], 
                                 text=top_routes1['Source airport']
                                ))
#другие не наносим, потому что они и так уже нанесены

fig.add_trace(go.Scattermapbox(mode = "lines",
                               hoverinfo='skip',
                               lat=top_routes1[top_routes1['Source airport'].\
                                               isin(top_routes1['Source airport'])]['Latitude_S'],
                               lon=top_routes1[top_routes1['Source airport'].\
                                               isin(top_routes1['Source airport'])]['Longitude_S'],
                              line = dict(color = 'lightpink', 
                                           #width=top_routes1['Intensity'].astype(int)
                               )))   

fig.add_trace(go.Scattermapbox(mode = "lines",
                               hoverinfo='skip',
                               lat=top_routes1[top_routes1['Destination airport'].\
                                               isin(top_routes1['Destination airport'])]['Latitude_D'],
                               lon=top_routes1[top_routes1['Destination airport'].\
                                               isin(top_routes1['Destination airport'])]['Longitude_D'],
                               line = dict(color = 'lightpink'
                                           #width= (top_routes1['Intensity'].astype(int))
                              )))   

title9 = 'Крупнейшие аэропорты и полёты между ними'

fig.update_layout(title = title9,mapbox_style="open-street-map")

fig.show()

#Ширину никак не получалось сделать зависимой от столбца Intensity